In [1]:
! apt-get install default-jre
!java -version
! pip install h2o
import h2o
h2o.init()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
openjdk version "11.0.5" 2019-10-15
OpenJDK Runtime Environment (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04)
OpenJDK 64-Bit Server VM (build 11.0.5+10-post-Ubuntu-0ubuntu1.118.04, mixed mode, sharing)
     |████████████████████████████████| 123.4MB 77kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=9146954f13075d589717b1e27465d81f2b8dae93782831ed96ad54ba489d594e
  Stored in directory: /root/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o
Checking whether there is an H2O instance running at h

H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_unknownUser_hv29nm
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [0]:
import h2o.grid

In [3]:
url =  "http://h2o-public-test-data.s3.amazonaws.com/smalldata/airlines/allyears2k_headers.zip"
data = h2o.import_file(url)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
train,valid,test=data.split_frame([0.8, 0.1], seed=7)

In [5]:
print("%d/%d/%d" %(train.nrows,valid.nrows,test.nrows))

35263/4265/4450


In [8]:
train.col_names

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'IsArrDelayed',
 'IsDepDelayed']

In [0]:
y = 'IsArrDelayed'

ignoreFields=['ArrDelay',
 'DepDelay','CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay','IsArrDelayed',
 'IsDepDelayed','ActualElapsedTime']

In [0]:
xAll=[i for i in train.names if i not in ignoreFields]
xLikely=['Month',
 'DayOfWeek','UniqueCarrier','Origin',
 'Dest','Distance','Cancelled','Diverted']

In [11]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator as hglm

glm = hglm(family='binomial')
glm.train(xAll, y, train, validation_frame = valid)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [14]:
glm.logloss(valid=True)

0.6206752592022771

In [17]:
g = h2o.grid.H2OGridSearch(
    hglm(
        family = 'binomial',
        lambda_search = True
    ),
    hyper_params = {
        'alpha': [x * 0.01 for x in range(100)]
    },
    search_criteria = {
        'strategy': 'RandomDiscrete',
        'max_models': 15,
        'max_runtime_secs': 50
    }
)

g.train(xAll, y, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [18]:
g 

      ...             logloss
0     ...  0.5878174495857804
1     ...  0.5890509638572097
2     ...  0.5891445538808737
3     ...  0.5915657102410856
4     ...  0.5929252056722528
5     ...  0.5932161601427512
6     ...  0.5956830125863718
7     ...  0.5985787396332599
8     ...  0.5992787993633355
9     ...   0.599659088905635
10    ...  0.6032131362236071
11    ...  0.6039144542444319
12    ...  0.6077068949971932

[13 rows x 4 columns]


In [19]:
g2 = h2o.grid.H2OGridSearch(
    hglm(
        family = 'binomial',
        lambda_search = True
    ),
    hyper_params = {
        'alpha': [0, 0.2, 0.4, 0.5, 0.6, 0.8, 0.99]
    },
    search_criteria = {
        'strategy': 'Cartesian',
    }
)

g2.train(xAll, y, train, validation_frame = valid)

glm Grid Build progress: |████████████████████████████████████████████████| 100%


In [20]:
g2 #Slightly worse performance

     ...             logloss
0    ...  0.5814858748809756
1    ...  0.5878174495857804
2    ...   0.589616733267399
3    ...  0.5915657102410856
4    ...  0.5927875536411129
5    ...  0.5943291272109674
6    ...  0.5992787993633355

[7 rows x 4 columns]
